<div style="background:#F5F7FA; height:100px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Want to do more?</span><span style="border: 1px solid #3d70b2;padding: 15px;float:right;margin-right:40px; color:#3d70b2; "><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
<span style="color:#5A6872;"> Try out this notebook with your free trial of IBM Watson Studio.</span>
</div>


# Machine learning optimization using the Cognitive Assistant component of spark-service

This notebook demonstrates the use of the Cognitive Assistant imported into a Python kernel and accessed via a Jupyter notebook. In this sample, you use the Cognitive Assistant package to optimize machine learning piplelines for a dataframe you create from a data set. Cognitive Assistant  helps you:
- Select the highest-performing pipelines for your dataframe, 
- Tune your chosen pipelines' hyperparameters for further optimization
- Generate python code for the optimized pipelines.  

The generated code assists you with subsequent training of models with your full data set (and subsequent updates), and with scoring (generating predictions for unlabeled data).

Cognitive assistant currently employs Apache Spark (PySpark) and SKLearn (Python scikit-learn) analytics on Spark 2.1 and Python 2.7, optionally with feature selection, for classification and regression.

In the first phase, when you invoke the `startOptimization()` method on your dataframe, Cognitive Assistant takes your data frame, samples it, and runs all pipelines with their default parameters, progressively allocating more data to the pipelines that cognitive assistant projects will have the best performance.  This approach allows Cognitive Assistant to quickly identify better-performing pipelines among many, therefore the user does not necessarily need to wait for the entire optimization to complete to select a preferred pipeline.

In the second phase, you invoke the `hyperParamOptimizer()` method on the chosen pipeline's ID.  A radial basis function black-box optimization is performed on the adjustable parameters of the pipeline, using your data frame.  The result is a new pipeline with a new pipeline ID.

In the third phase, you can generate code for your optimized pipeline and populate a notebook cell with it.  Currently, code generation is supported for sklearn classifiers.

The notebook remains interactive during the optimizations. The execution counter and the kernel activity indicator reflect the activity of both the notebook user and the Cognitive Assistant.

## Contents

This notebook has the following main sections:
1. [One-time setup: Load or update packages](#update)
1. [Download the data](#download)
1. [Format the data for PySpark ML](#format)
1. [Run the cognitive assistant to rank ML pipelines using your dataset, and monitor its progress](#cads)
1. [Use cognitive assistant to optimize hyperparameters of a chosen pipeline for your dataset](#hpo)
1. [Obtain readable/executable code for your chosen pipeline](#readableCode)
1. [Stop cognitive assistant](#shutDown)




<a id="update"></a>
## 1. One-time setup: update sklearn >0.17 and xgboost >= 0.6
This need only be done one time per user.  The packages are stored in your client's home directory under ~/.local

You should restart your kernel (**Kernel** > **Restart**) if any of these packages are installed or updated.

In [1]:
!pip install -q --user -U sklearn

In [2]:
!pip install -q --user -U xgboost

In [3]:
!pip install -q --user -U numpy

Reminder: restart your kernel (**Kernel** > **Restart**) if **any** of these packages were not already up-to-date, then continue with the next section.

<a id="download"></a>
## 2. Download the data (one time per user, per dataset)

Because Cognitive Assistant is especially suited to large data sets, in the following step, you'll be retrieving a *portion of a*  large (2.6 GB) .csv file that contains data on HIGGS boson particles. You can modify the cell to get more of or all of the file.

From the description of the data set on the UCI Web site: "The data has been produced using Monte Carlo simulations. The first 21 features (columns 2-22) are kinematic properties measured by the particle detectors in the accelerator. The last seven features are functions of the first 21 features; these are high-level features derived by physicists to help discriminate between the two classes. There is an interest in using deep learning methods to obviate the need for physicists to manually develop such features. Benchmark results using Bayesian Decision Trees from a standard physics package and 5-layer neural networks are presented in the original paper. The last 500,000 examples are used as a test set."

In [4]:
%%bash
#Download a portion of a popular compressed csv dataset.  The target kilobytes of data are specified after '-ge' nine or so lines below
cat << 'EOF' > limited.sh
# optionally you can limit the download rate to wget via --limit-rate=2m
# limiting the rate provides better control of the total amount of data that is downloaded
wget -nv  -O limited.csv.gz --limit-rate=2m https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz &
WGET_PID=$!
while [ `ps $WGET_PID | wc -l ` -eq 2 ] ; do
    sleep 1
    SIZE=`du -k --apparent-size limited.csv.gz | awk '{print \$1}'`
    echo $SIZE kilobytes downloaded
    #modify the target kilobyte download size after -ge below, or comment out the line below to get the whole file
    if [ -e limited.csv.gz ] ; then if [ $SIZE -ge 3500  ] ; then kill -15 $WGET_PID; fi; fi
done
EOF
cat limited.sh

# optionally you can limit the download rate to wget via --limit-rate=2m
# limiting the rate provides better control of the total amount of data that is downloaded
wget -nv  -O limited.csv.gz --limit-rate=2m https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz &
WGET_PID=$!
while [ `ps $WGET_PID | wc -l ` -eq 2 ] ; do
    sleep 1
    SIZE=`du -k --apparent-size limited.csv.gz | awk '{print \$1}'`
    echo $SIZE kilobytes downloaded
    #modify the target kilobyte download size after -ge below, or comment out the line below to get the whole file
    if [ -e limited.csv.gz ] ; then if [ $SIZE -ge 3500  ] ; then kill -15 $WGET_PID; fi; fi
done


In [5]:
!echo "expect an unexpected-end-of-file-message, that is ok if you specified a limited download size"
!bash limited.sh
#an incomplete last line will likely occur in the case of a partial download, and must be discarded
!gunzip -c -q limited.csv.gz | head -n -2 > limited.csv
!echo `wc -l limited.csv` ' complete lines of csv retrieved'

expect an unexpected-end-of-file-message, that is ok if you specified a limited download size
1544 kilobytes downloaded
3656 kilobytes downloaded
limited.sh: line 12:  3748 Terminated              wget -nv -O limited.csv.gz --limit-rate=2m https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz

gzip: limited.csv.gz: unexpected end of file
14614 limited.csv  complete lines of csv retrieved


<a id="format"></a>
## 3. Format the data

Format the data into a Pyspark SQL data frame consisting of a numeric `label` column and a vector `features` column.

In [6]:
version_list = sc.version.split('.')
major_version = version_list[0]
if int(major_version) == 1:
    from pyspark.mllib.linalg import Vectors
else:
    from pyspark.ml.linalg import Vectors

from pyspark.ml.feature import VectorAssembler

if int(major_version) == 1:
    raise Warning("Spark versions before 2.0 may no longer function with this notebook. Use Kernel... Change Kernel... to select Spark 2.0 ")

In [7]:
df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='false', inferschema='true')\
  .load("limited.csv")

In [8]:
allNames = [f.name for f in df.schema.fields]
#FIRST column is the label (unlike many csv where it is the last)
labelName = allNames[0]
print 'labelName ' + labelName
featureNames = [ f.name for f in df.schema.fields if f.name != labelName ]
print 'featureNames=' + str(featureNames)

labelName _c0
featureNames=['_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28']


In [9]:
featureAssembler = VectorAssembler(
    inputCols=featureNames,
    outputCol="features")

In [10]:
#create features (vector) column then select label and features column, with renaming.
inDF=featureAssembler.transform(df).selectExpr(labelName + " as label","features as features")


In [11]:
print inDF

DataFrame[label: double, features: vector]


### 3.1 Remove compressed file

You can clear up space on your filesystem by removing the compressed *limited.csv.gz* file that you downloaded. Because of the deferred computation in Spark, the uncompressed file is accessed later and cannot be erased at this point.

In [12]:
!rm -f limited.csv.gz

<a id="cads"></a>
## 4. Run the Cognitive Assistant optimization to rank machine learning pipelines for your data

Before you can use the Cognitive Assistant, you must import the `cognitive_assistant` package and then start the assistant.

In [13]:
import cognitive_assistant as cognitive

cognitive_assistant has instantiated 'assistant'


In [14]:
#override CADS.conf hyperparameter optimization defaults for demonstration purposes
cadsArgs='-DBBO_MAX_WALL_CLOCK_TIME=300'
#better values for hyperparameter optimization, but will HPO will take too long for a demonstration
#cadsArgs='-DBBO_MAX_WALL_CLOCK_TIME=10000 -DBBO_MAX_EVALUATIONS=350'
#or use CADS.conf defaults:
#cadsArgs=''
cognitive.assistant.startAssistant(cadsArgs=cadsArgs)

Please wait...Cognitive Assistant is starting
WARNING your arguments to startOptimization must be compatible with the cadsArgs you have supplied here.
mario is available and responding
mario is available and responding
cads is available and responding
ok to proceed
Assistant is started.
If you have directly invoked startAssistant() or have just imported cognitive_assistant, you may now invoke startOptimization(<dataFrame>).


### 4.1 Start the Cognitive Assistant optimization

You invoke the Cognitive Assistant function by providing it the following information:

-  The name of the data frame
-  The prediction type (default is 'CLASSIFICATION')
    -  Alternatively, specify data frame name and predictionType='REGRESSION'


In [15]:
cognitive.assistant.startOptimization(inDF)

Assistant is determining the optimum pipeline for dataFrame NoNameSpecified with row count= 14614 in background.
To view progress, invoke visualizeProgress().  To stop the optimization and erase the results, invoke stopAssistant().


### 4.2 Monitor the ranking of pipelines as optimization progresses

This code defines a dynamically-updated leaderboard to monitor the progress of optimization in your notebook.


In [16]:
cognitive.assistant.leaderboard()

ID,Approach,FSCORE [Loss],Data Allocation Pct
P26,[TH Pending] Data -> FS: SklearnJupyterPercentile -> Sampler -> CL: JupyterXgboost,0.6626 [33.64%],6.84
P3,[TH Pending] Data -> FS: SklearnJupyterPercentile -> Sampler -> CL: SklearnJupyterGradientBoostedTree,0.6574 [34.15%],6.84
P30,[TH Pending] Data -> FS: SklearnJupyterPercentile -> Sampler -> CL: PySpark JupyterRandom Forest,0.6459 [35.26%],6.84
P31,[TH Pending] Data -> FS: SklearnJupyterPercentile -> Sampler -> CL: SklearnJupyterDecision Tree [Explainable],0.6083 [38.44%],6.84
P4,[TH Pending] Data -> FS: SklearnJupyterPercentile -> Sampler -> CL: SklearnJupyterLogistic Regression L2 [Explainable],0.6001 [39.33%],6.84
P6,[TH Pending] Data -> FS: -> Sampler -> CL: SklearnJupyterLogistic Regression L2 [Explainable],0.5916 [40.29%],6.84
P9,[TH Pending] Data -> FS: SklearnJupyterPercentile -> Sampler -> CL: PySpark JupyterLogistic Regression [Explainable],0.5884 [39.45%],6.84
P29,[TH Pending] Data -> FS: SklearnJupyterPercentile -> Sampler -> CL: SklearnJupyterKNearestNeighbors,0.5714 [42.73%],6.84
P8,[TH Pending] Data -> FS: SklearnJupyterPercentile -> Sampler -> CL: SklearnJupyterNaive Bayes,0.5571 [41.2%],6.84
P28,[TH Pending] Data -> FS: SklearnJupyterPercentile -> Sampler -> CL: SklearnJupyterPerceptron,0.4852 [42.74%],6.84


In [17]:
#Instruct the leaderboard to stop before the next automatic query/update.  It can be restarted at any time (after the query interval).
#cognitive.assistant.leaderboard(stop=True)

<a id="leaderboard"></a>

In [18]:
# raise an exception so that in-order all-cell execution is halted at this cell.  Instruct the user to proceed on a cell-by-cell basis.
raise Warning("Stopping 'Run All' execution at this point to allow time for optimization to progress.  Execute subsequent cells one-at-a-time as needed. ")

Warning: Stopping 'Run All' execution at this point to allow time for optimization to progress.  Execute subsequent cells one-at-a-time as needed. 

### Generate performance curves for all pipelines.  Re-invoke this as many times as you want to update and view current results. An error message can be expected while the optimization is starting up.

In [139]:
cognitive.assistant.visualizeProgress()

### 4.3 Do a quick check of CPU - Optional

To further check on the progress of your optimization, you can can run the following command to display CPU activity and the number of processes that are running.

In [158]:
!top -u $USER -n 1

=top - 07:28:03 up 26 days, 10:08,  0 users,  load average: 7.71, 6.58, 6.33
Tasks: 948 total,   2 running, 946 sleeping,   0 stopped,   0 zombie
%Cpu(s): 10.5 us,  4.5 sy,  0.0 ni, 84.9 id,  0.0 wa,  0.0 hi,  0.1 si,  0.0 st
KiB Mem : 39597036+total, 25603824+free, 66286080 used, 73646048 buff/cache
KiB Swap:  1048572 total,  1048572 free,        0 used. 32667980+avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND     
 5289 sf9b-79+  20   0 10.878g 523112  13064 S  96.4  0.1   1:40.14 java        
 2146 sf9b-79+  20   0 13.985g 859076  13316 S  64.3  0.2   2:14.45 java        
 1823 sf9b-79+  20   0 3915624 264128  18756 S  57.1  0.1   0:36.10 python      
 5367 sf9b-79+  20   0 11.432g 335832  13208 S  21.4  0.1   0:44.16 java        
10672 sf9b-79+  20   0  160624   3048   1532 R  10.7  0.0   0:00.08 top         
10660 sf9b-79+  20   0  425060  18656   5644 S   7.1  0.0   0:00.23 python      
39084 sf9b-79+  20   0 7342756 204884   7620 S   7.1 

### 4.4 Continue your work!

Cognitive Assistant is working in the background so you can continue to use your notebook interactively as the optimization progresses. This can include additional optimization and code generation as detailed below:


<a id="hpo"></a>
## 5 Optimize hyperparameters for your chosen pipeline

If you like the evolving performance evaluation of a pipeline, you do not need to wait for CADS ranking of ML algorithms to complete. You can ask Cognitive Assistant to optimize the hyperparameters for your chosen pipeline (identified by pipeline ID `P<nn>`) at any time. Black-Box Optimization can take considerable time, which can be controlled by the startup parameters first given to the `startAssistant()` method.

In [189]:
cognitive.assistant.hyperParamOptimizer(pipelineID='P2')

You can periodically return to the `visualizeProgress()` method to see the status of the HPO.  The original pipeline will have the additional label "Optimizing" while the optimization is in progress and the new pipeline will be labeled "Optimized" when HPO is complete.

<a id="readableCode"></a>
## 6 Obtain "readable code" for your chosen pipeline

To deploy the pipeline outside of Cognitive Assistant, you can request a "readable code" representation of the pipeline identified by ID.  Remember that the hyperparameter-optimized pipeline is given its own pipeline ID, should you want the generated code to match its parameterization. This code is emitted directly into the notebook cell for you, the user, to use as you please. The query is done asynchronously in a two-step process in separate notebook cells to avoid deadlocking the kernel.

The emitted code is intended to run with minimum necessary user modifications (you will need to provide the data frame names you wish to operate upon). Other minor fixups may be necessary.  Currently in cognitive-assistant, "readable code" output is limited to pipelines with sklearn classifiers.

In [198]:
#fixups for generated code to run as-is in this notebook.  Modify to suit.
gini='gini'
trainDF=inDF

In [203]:
cell=cognitive.assistant.generateCode('P41')

no json returned for pipeline (AlgorithmTrajectoryUUID): P41


In [228]:
#if the code was not emitted above, invoke the following:
cell.generateCell()

no json returned for pipeline (AlgorithmTrajectoryUUID): P41


In [238]:
# code for pipeline P41
from numpy import array as nparray
from pyspark.sql import Row
from pyspark.sql import SQLContext
def np2df(npY,npX):
       rddResult = sc.parallelize(zip(npY, npX))
       dfResult = sqlContext.createDataFrame(rddResult.map(lambda r:Row(label=float(r[0]), features=Vectors.dense(r[1]))))     
       return dfResult;

def df2npF(df, featuresCol="features"):
        features = df.select(featuresCol)
        features2 = features.withColumnRenamed(featuresCol, "my_column")
        X = nparray([f.my_column.toArray() for f in features2.collect()])
        return X;

def df2npL(df, labelCol="label"):
        label = df.select(labelCol)
        Y = nparray([l.label for l in label.collect()])
        return Y;

from sklearn.ensemble import GradientBoostingClassifier
clf =  GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, init=None, random_state=None, max_features=1.0, verbose=0, max_leaf_nodes=None, warm_start=False)
clf.fit(df2npF(df=trainDF), df2npL(df=trainDF))



GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=1.0, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [564]:
cell2=cognitive.assistant.generateCode('P2')

no json returned for pipeline (AlgorithmTrajectoryUUID): P2


In [565]:
#if the code was not emitted above, invoke the following:
cell2.generateCell()

no json returned for pipeline (AlgorithmTrajectoryUUID): P2


In [566]:
# code for pipeline P2
from numpy import array as nparray
from pyspark.sql import Row
from pyspark.sql import SQLContext
def np2df(npY,npX):
       rddResult = sc.parallelize(zip(npY, npX))
       dfResult = sqlContext.createDataFrame(rddResult.map(lambda r:Row(label=float(r[0]), features=Vectors.dense(r[1]))))     
       return dfResult;

def df2npF(df, featuresCol="features"):
        features = df.select(featuresCol)
        features2 = features.withColumnRenamed(featuresCol, "my_column")
        X = nparray([f.my_column.toArray() for f in features2.collect()])
        return X;

def df2npL(df, labelCol="label"):
        label = df.select(labelCol)
        Y = nparray([l.label for l in label.collect()])
        return Y;

from sklearn.ensemble import GradientBoostingClassifier
clf =  GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, init=None, random_state=None, max_features=1.0, verbose=0, max_leaf_nodes=None, warm_start=False)
clf.fit(df2npF(df=trainDF), df2npL(df=trainDF))



GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=1.0, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [570]:
# code for pipeline P2
from numpy import array as nparray
from pyspark.sql import Row
from pyspark.sql import SQLContext
def np2df(npY,npX):
       rddResult = sc.parallelize(zip(npY, npX))
       dfResult = sqlContext.createDataFrame(rddResult.map(lambda r:Row(label=float(r[0]), features=Vectors.dense(r[1]))))     
       return dfResult;

def df2npF(df, featuresCol="features"):
        features = df.select(featuresCol)
        features2 = features.withColumnRenamed(featuresCol, "my_column")
        X = nparray([f.my_column.toArray() for f in features2.collect()])
        return X;

def df2npL(df, labelCol="label"):
        label = df.select(labelCol)
        Y = nparray([l.label for l in label.collect()])
        return Y;

from sklearn.ensemble import GradientBoostingClassifier
clf =  GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, init=None, random_state=None, max_features=1.0, verbose=0, max_leaf_nodes=None, warm_start=False)
clf.fit(df2npF(df=trainDF), df2npL(df=trainDF))



GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=1.0, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [572]:
# code for pipeline P2
from numpy import array as nparray
from pyspark.sql import Row
from pyspark.sql import SQLContext
def np2df(npY,npX):
       rddResult = sc.parallelize(zip(npY, npX))
       dfResult = sqlContext.createDataFrame(rddResult.map(lambda r:Row(label=float(r[0]), features=Vectors.dense(r[1]))))     
       return dfResult;

def df2npF(df, featuresCol="features"):
        features = df.select(featuresCol)
        features2 = features.withColumnRenamed(featuresCol, "my_column")
        X = nparray([f.my_column.toArray() for f in features2.collect()])
        return X;

def df2npL(df, labelCol="label"):
        label = df.select(labelCol)
        Y = nparray([l.label for l in label.collect()])
        return Y;

from sklearn.ensemble import GradientBoostingClassifier
clf =  GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, init=None, random_state=None, max_features=1.0, verbose=0, max_leaf_nodes=None, warm_start=False)
clf.fit(df2npF(df=trainDF), df2npL(df=trainDF))



GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=1.0, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [574]:
# code for pipeline P2
from numpy import array as nparray
from pyspark.sql import Row
from pyspark.sql import SQLContext
def np2df(npY,npX):
       rddResult = sc.parallelize(zip(npY, npX))
       dfResult = sqlContext.createDataFrame(rddResult.map(lambda r:Row(label=float(r[0]), features=Vectors.dense(r[1]))))     
       return dfResult;

def df2npF(df, featuresCol="features"):
        features = df.select(featuresCol)
        features2 = features.withColumnRenamed(featuresCol, "my_column")
        X = nparray([f.my_column.toArray() for f in features2.collect()])
        return X;

def df2npL(df, labelCol="label"):
        label = df.select(labelCol)
        Y = nparray([l.label for l in label.collect()])
        return Y;

from sklearn.ensemble import GradientBoostingClassifier
clf =  GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, init=None, random_state=None, max_features=1.0, verbose=0, max_leaf_nodes=None, warm_start=False)
clf.fit(df2npF(df=trainDF), df2npL(df=trainDF))



GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=1.0, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

<a id="shutdown"></a>
## 6. Stop the cognitive assistant process

When optimization has completed or progressed to an acceptable point, the cognitive assistant can be stopped by running the following command:

In [577]:
cognitive.assistant.stopAssistant()

## begin stub
lf_filename = '/gpfs/global_fs01/sym_shared/YPProdSpark/user/sf9b-795b2b888c32b6-772f4e1cd93d/notebook/cads-72bc0314-3bc0-4c98-9114-1a193248e823/mario.output/script/meta/sfa1968aa04004a6c4ab7827076db72ed.json'
lf_input_test_df_id = 's7fa24ea67d79743be40255f3eeed3471_test_df'
lf_input_train_df_id = 's7fa24ea67d79743be40255f3eeed3471_train_df'
lf_problem_type = 'binary-classification'
lf_thread_id = 'sfa1968aa04004a6c4ab7827076db72ed_thread'
from spark_ml_wrapper.evaluate_analytics_performance import aperf_go
aperf_go(train_input_dfi=lf_input_train_df_id,
         test_input_dfi=lf_input_test_df_id,
         thi=lf_thread_id,
         output_filename=lf_filename,
         problem_type=lf_problem_type)
## end stub

Instance CADS services successfully terminated by stopAssistant().  Therefore terminating skulker:
Assistant is stopped.  To start anew, invoke startAssistant(). Invoking import again will not re-start cognitive_assistant.


## Summary

You downloaded and formatted a publicly available data set and then used the cognitive assistant to identify a high-performance pipeline and adjust its hyperparameters for your dataset.  You then used cognitive-assistant to generate code (this cognitive-assistant feature is currently limited to sklearn classifiers) so that you can invoke these pipelines independently of cognitive-assistant. Congratulations!

## Authors

**Peter D. Kirchner**, PhD (Electrical Engineering), is a Research Scientist persuing computer science research in machine learning and cloud computing at the IBM Thomas J. Watson Research Center. He is presently engaged in cognitive automation of data science workflow to assist data scientists, focused on cloud-based deployments and scalability.

**Gregory Bramble** is a Research Software Engineer at the IBM T.J. Watson Research Center. Greg has 10+ years development experience in industry and research, and has built several applications which have achieved widespread use. Greg studied Computer Science and Mathematics at Columbia University where his advanced coursework included AI, Graph Theory, and Algorithms.

**Mike Sochka** is a content designer focusing on Watson Studio and Watson Machine Learning. 

***
### References

Ashish Sabharwal, Horst Samulowitz, Gerald Tesauro, "Selecting Near-Optimal Learners via Incremental Data Allocation". Proc. 30th AAAI pp. 2007-15 (2016) http://dl.acm.org/citation.cfm?id=3016179

Biem, A., Butrico, M., Feblowitz, M., Klinger, T., Malitsky, Y., Ng, K., Perer, A., and Reddy, C., Riabov, A., Samulowitz, H., Sow, D., Tesauro, G., Turaga, D. "Towards Cognitive Automation of Data Science".  Proc. 29th AAAI pp. 4268-9 (2015) http://dl.acm.org/citation.cfm?id=2888116.2888360

Costa, A., Nannicini, G. "RBFOpt: an open-source library for black-box optimization with costly function evaluations" Optimization Online https://pdfs.semanticscholar.org/6bdb/d2de4988fab315bc7e6ec0cd14ed3ae8c018.pdf (2014)

Baldi, P., P. Sadowski, and D. Whiteson. “Searching for Exotic Particles in High-energy Physics with Deep Learning.” Nature Communications 5 (July 2, 2014).

Lichman, M. (2013). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

 [If you landed here from 'Run All' cells, you can navigate to the leaderboard](#leaderboard)